In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/Диплом

all_sent.pickle  CO_fold  labels.pickle  transformer_model


In [ ]:
!pip install  pyyaml==5.4.1 sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 636 kB 4.5 MB/s 
     |████████████████████████████████| 79 kB 4.4 MB/s 
     |████████████████████████████████| 4.2 MB 32.1 MB/s 
     |████████████████████████████████| 1.2 MB 43.1 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 51.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=d0eabce7db5f9a1c3af75d4b6819943dda56b28bdd0a3df819b4ebcea6417f60
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import warnings
import gc
import pickle

import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE

import gensim
import gensim.models as g
import gensim.downloader

from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

from sentence_transformers import SentenceTransformer

import tensorflow_hub as hub

# Манипулирование данными
import pandas as pd # для манипулирования данными

# Визуализация
import plotly.express as px # для визуализации данных
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
def vectorize_sentence(sentence,model):
    nlp = English()
    tokenizer = Tokenizer(nlp.vocab)
    a = []
    for i in tokenizer(sentence):
        try:
            a.append(model.get_vector(str(i)))
        except:
            pass
    a=np.array(a).mean(axis=0)
    a = np.zeros(300) if np.all(a!=a) else a
    return a
def tsne_training(X, n_components=2, perplexity=30):
    embed = TSNE(perplexity=perplexity, n_components=n_components, random_state=33, n_iter=10000, n_jobs=-1)
    X_embedded = embed.fit_transform(X)
    # Вывод результатов
    print(perplexity)
    print('New Shape of X: ', X_embedded.shape)
    print('Kullback-Leibler divergence after optimization: ', embed.kl_divergence_)
    print('No. of iterations: ', embed.n_iter_)
    #вывод('Embedding vectors: ', embed.embedding_)
    return X_embedded, embed.kl_divergence_
def plotting(X_embedded, labels):
    # Создание диаграммы разброса
    fig = px.scatter(None, x=X_embedded[:,0], y=X_embedded[:,1], 
                     labels={
                         "x": "Dimension 1",
                         "y": "Dimension 2",
                     },
                     opacity=1, color=np.array(labels).astype(str))

    # Изменение цвета фона графика
    fig.update_layout(dict(plot_bgcolor = 'white'))
    # Обновление линий осей
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                     zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                     showline=True, linewidth=1, linecolor='black')

    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                     zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                     showline=True, linewidth=1, linecolor='black')
    # Установка названия рисунка
    fig.update_layout(title_text="t-SNE")
    # Обновление размера маркера
    fig.update_traces(marker=dict(size=3))
    fig.show()
def plotting_3d(X_embedded, labels):
    # Создание диаграммы разброса
    fig = px.scatter_3d(None, x=X_embedded[:,0], y=X_embedded[:,1],
                       z=X_embedded[:,2],
                       labels={
                           "x": "Dimension 1",
                           "y": "Dimension 2",
                           "z": "Dimension 2",
                       },
                       opacity=1, color=np.array(labels).astype(str))

    # Изменение цвета фона графика
    fig.update_layout(dict(plot_bgcolor = 'white'))
    # Обновление линий осей
    fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                     zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                     showline=True, linewidth=1, linecolor='black')

    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                     zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                     showline=True, linewidth=1, linecolor='black')
    fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')
    # Установка названия рисунка
    fig.update_layout(title_text="t-SNE")
    # Обновление размера маркера
    fig.update_traces(marker=dict(size=3))
    fig.show()
def get_graphics(X, labels):
    act_perp = 5
    X_embed,diver = tsne_training(X, perplexity=5)
    X_embed1,diver1 = tsne_training(X, perplexity=30)
    if diver1 < diver:
        X_embed = X_embed1
        diver = diver1
        act_perp = 30
    del X_embed1, diver1
    gc.collect()
    X_embed1,diver1 = tsne_training(X, perplexity=50)
    if diver1 < diver:
        X_embed = X_embed1
        diver = diver1
        act_perp = 50
    del X_embed1, diver1
    gc.collect()
    print('Minimal Kullback-Leibler divergence perplexity 2d = ', act_perp)
    print('Minimal Kullback-Leibler divergence 2d = ', diver)
    plotting(X_embed, labels)
    del X_embed, diver
    gc.collect()
    print("______________")
    print("3D Results:")
    X_embed, diver = tsne_training(X, n_components=3, perplexity=act_perp)
    plotting_3d(X_embed, labels)

### Загрузка переменных

In [ ]:
with open('/content/drive/MyDrive/Диплом/all_sent.pickle', 'rb') as f:
     all_sent = pickle.load(f)
with open('/content/drive/MyDrive/Диплом/labels.pickle', 'rb') as f:
     labels = pickle.load(f)

### Global Vector(GloVE)

In [ ]:
gv = gensim.downloader.load('glove-wiki-gigaword-300') #376mb
X = [vectorize_sentence(i, model=gv) for i in all_sent]
del gv
gc.collect()

[==================================================] 100.0% 376.1/376.1MB downloaded


0

In [ ]:
get_graphics(X, labels)

5
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.4902571439743042
No. of iterations:  9999
30
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.3433809280395508
No. of iterations:  9999
50
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.1897226572036743
No. of iterations:  9999
Minimal Kullback-Leibler divergence perplexity 2d =  50
Minimal Kullback-Leibler divergence 2d =  1.1897226572036743


______________
3D Results:
50
New Shape of X:  (3991, 3)
Kullback-Leibler divergence after optimization:  1.1587400436401367
No. of iterations:  1099


### Word2Vec

In [ ]:
word2vec = gensim.downloader.load('word2vec-google-news-300') #1.66 gb
X1 = [vectorize_sentence(i, model=word2vec) for i in all_sent]
del word2vec
gc.collect()

[==================================================] 100.0% 1662.8/1662.8MB downloaded


0

In [ ]:
get_graphics(X1, labels)

5
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.5719420909881592
No. of iterations:  9999
30
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.3509117364883423
No. of iterations:  9999
50
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.236261010169983
No. of iterations:  9999
Minimal Kullback-Leibler divergence perplexity 2d =  50
Minimal Kullback-Leibler divergence 2d =  1.236261010169983


______________
3D Results:
50
New Shape of X:  (3991, 3)
Kullback-Leibler divergence after optimization:  0.9976276159286499
No. of iterations:  9999


### BERT

In [ ]:
bert = SentenceTransformer('stsb-roberta-large') #1.3 gb
X2 = [bert.encode(i) for i in all_sent]
del bert
gc.collect()

Downloading:   0%|          | 0.00/748 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/191 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

822

In [ ]:
get_graphics(X2, labels)

MathBERT

In [ ]:
bert = SentenceTransformer('tbs17/MathBERT') #1.3 gb
X2 = [bert.encode(i) for i in all_sent]
del bert
gc.collect()

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/tbs17_MathBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


421

In [ ]:
get_graphics(X2, labels)

5
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.3677211999893188
No. of iterations:  9999
30
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.6064136028289795
No. of iterations:  9999
50
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.536631464958191
No. of iterations:  9999
Minimal Kullback-Leibler divergence perplexity 2d =  5
Minimal Kullback-Leibler divergence 2d =  1.3677211999893188


______________
3D Results:
5
New Shape of X:  (3991, 3)
Kullback-Leibler divergence after optimization:  1.2273260354995728
No. of iterations:  9999


### Universal Sentence Encoder

In [ ]:
universal = hub.load("./transformer_model")
X3 = np.vstack(np.array([universal([i]) for i in all_sent]))
gc.collect()

In [ ]:
get_graphics(X3, labels)

# Модели с параметрами

### Tf-Idf ngram=(1,4) df =.8

In [ ]:
tfidf= TfidfVectorizer(min_df=5,max_df=.8, ngram_range=(1, 4)) 
X4 = tfidf.fit_transform(all_sent)
del tfidf
gc.collect()

813

In [ ]:
get_graphics(X4, labels)

5
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.4870058298110962
No. of iterations:  9999
30
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.4731581211090088
No. of iterations:  9999
50
New Shape of X:  (3991, 2)
Kullback-Leibler divergence after optimization:  1.3137896060943604
No. of iterations:  9999
Minimal Kullback-Leibler divergence perplexity 2d =  50
Minimal Kullback-Leibler divergence 2d =  1.3137896060943604


______________
3D Results:
50
New Shape of X:  (3991, 3)
Kullback-Leibler divergence after optimization:  1.0714268684387207
No. of iterations:  9999


### Bag of words ngram=(1,5) df =.99

In [ ]:
bow= CountVectorizer(min_df=5,max_df=.99, ngram_range=(1, 5)) 
X4 = bow.fit_transform(all_sent)
del tfidf
gc.collect()

In [ ]:
get_graphics(X4, labels)